In [3]:
import xarray
import intake
import numpy as np

In [ ]:
number_in_cluster = 10

%run start_cluster.ipynb 
client = get_dask_cluster(number_in_cluster)
client

In [5]:
catalog = intake.open_catalog("s3://noaa-nos-cora-pds/CORA_intake.yml",storage_options={'anon':True})
print(list(catalog))

ds = catalog['CORA-V1-fort.63'].to_dask()


ds

['CORA-V1-fort.63', 'CORA-V1-maxele.63', 'CORA-V1-fort.64', 'CORA-V1-500m-grid-1979-2022']


<xarray.Dataset>
Dimensions:      (mesh: 1, node: 1813443, nele: 3564104, nvertex: 3, nbou: 186,
                  nope: 1, neta_dim: 186, nvel_dim: 62972, time: 385704)
Coordinates:
  * time         (time) datetime64[ns] 1979-01-01 ... 2022-12-31T23:00:00
    x            (node) float64 dask.array<chunksize=(160000,), meta=np.ndarray>
    y            (node) float64 dask.array<chunksize=(160000,), meta=np.ndarray>
Dimensions without coordinates: mesh, node, nele, nvertex, nbou, nope,
                                neta_dim, nvel_dim
Data variables: (12/14)
    adcirc_mesh  (mesh) int32 dask.array<chunksize=(1,), meta=np.ndarray>
    depth        (node) float64 dask.array<chunksize=(160000,), meta=np.ndarray>
    element      (nele, nvertex) int32 dask.array<chunksize=(3564104, 3), meta=np.ndarray>
    ibtype       (nbou) int32 dask.array<chunksize=(186,), meta=np.ndarray>
    ibtypee      (nope) int32 dask.array<chunksize=(1,), meta=np.ndarray>
    max_nvdll    float64 ...
    ...           ...
    nbvv         (nvel_dim) int32 dask.array<chunksize=(62972,), meta=np.ndarray>
    neta         float64 ...
    nvdll        (nope) int32 dask.array<chunksize=(1,), meta=np.ndarray>
    nvel         float64 ...
    nvell        (nbou) int32 dask.array<chunksize=(186,), meta=np.ndarray>
    zeta         (time, node) float64 dask.array<chunksize=(240, 160000), meta=np.ndarray>
Attributes: (12/49)
    Conventions:        UGRID-0.9.0
    NCO:                netCDF Operators version 4.7.5 (Homepage = http://nco...
    a00:                0.0
    agrid:              NOMAD mesh v1e MSL - fixed bahamas bathy
    b00:                1.0
    c00:                0.0
    ...                 ...
    slam0:              265.5
    source:             RENCI_HPC
    statim:             0.0
    tau0:               -3.0
    title:              NOAA-Reanalysis-Post
    version:            noaa.stofs.2d.glo.v1.1.0r2-21-g7052115

In [ ]:
lon_min = -76
lon_max = -62
lat_min = 30.5
lat_max = 35
min_time = '1980-07-01'
max_time = '1980-9-01'




# file to write the output to. 
output_file_name = cat_entry+'_subset.nc'

#  all_triangle_nodes_in_bbox : True/False
#  True means only export if all nodes of the triangle are in the bounding box
#  False means export all nodes of a triangle if any node is in the bounding box
all_triangle_nodes_in_bbox = True # True/False 



In [ ]:
%%time
# Subset the data by time and AOI
# Write the resulting data to a local netCDF file

e = ds.element.values.astype(int) - 1
dsx_e = ds.x.values[e]
dsy_e = ds.y.values[e]

lon_in_bounds = np.logical_and(dsx_e>=lon_min,dsx_e<=lon_max)

lat_in_bounds = np.logical_and(dsy_e>=lat_min, dsy_e<=lat_max)

if all_triangle_nodes_in_bbox:
    lats_in = np.all(lat_in_bounds,axis=1)
    lons_in = np.all(lon_in_bounds,axis=1)
else:
    lats_in = np.any(lat_in_bounds,axis=1)
    lons_in = np.any(lon_in_bounds,axis=1)   
element_mask = (lons_in & lats_in)
extract_nodes = np.where(element_mask)[0]
unique_nodes = np.unique(e[extract_nodes])
print(unique_nodes)
print("about to subset")
# subset based on AOI and time frame
ds_u =ds.sel(time=slice(min_time,max_time),
             node=unique_nodes,nele=extract_nodes).compute()

print("Nodes: ",len(unique_nodes), " Timesteps: ",ds_u.dims['time'])
new_e_updated=np.searchsorted(unique_nodes,e[extract_nodes])
ds_u['element'].data = new_e_updated + 1
ds_u.to_netcdf(cat_entry+'_subset.nc',engine="netcdf4")
ds_u.close
ds.close()
print("Wrote subset to: ", output_file_name)

In [ ]:
del ds

In [ ]:
# Plot data from new subset file
import xarray
import matplotlib.pyplot as plt
ds = xarray.open_dataset(output_file_name)
time_index = 0
plt.scatter(ds.x,ds.y,c=ds.zeta[time_index,:],cmap="RdYlBu",marker=",")
ds.close()



In [ ]:
# Shutdown the dask gateway
from dask_gateway import Gateway
gateway = Gateway()
for c in gateway.list_clusters():
    print(c.name)
    gateway.connect(c.name).close(shutdown=True)